In [ ]:
import time
import importlib
import cv2
import numpy as np
from ugot import ugot
import pose_yolo
import face_rec
importlib.reload(pose_yolo)
importlib.reload(face_rec)
from pose_yolo import run_pose_control_inline
from IPython.display import display, clear_output, Image

got = ugot.UGOT()
got.initialize('192.168.1.217')

got.load_models(['word_recognition', 'color_recognition', 'gesture',
                 'line_recognition', 'face_recognition', 'face_attribute'])

got.set_track_recognition_line(0)

got.open_camera()

print("Done!")

### Inbuilt UGOT models

- Line following
- Color recognition
- Text recognition

In [ ]:
def line_follow(mult=0.25, speed=35):
    offset, type, x, y = got.get_single_track_total_info()
    rotation_speed = int(offset * mult)

    got.mecanum_move_xyz(x_speed=0, y_speed=speed, z_speed=rotation_speed)

    return type, x, y

def color_rec():
    color_info = got.get_color_total_info()

    frame = got.read_camera_data()
    if frame is not None:
        nparr = np.frombuffer(frame, np.uint8)
        data = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if color_info[0]:
            cv2.putText(data, color_info[0], (50, 40), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            c_x = color_info[2]
            c_y = color_info[3]
            h = color_info[4]
            w = color_info[5]

            cv2.rectangle(data, (int(c_x - w / 2), int(c_y - h / 2)), 
                            (int(c_x + w / 2), int(c_y + h / 2)), (0, 0, 255), 2)
            

        _, jpeg = cv2.imencode('.jpg', data)
        clear_output(wait=True)
        display(Image(data=jpeg.tobytes()))

        return color_info[0]


def text_rec():
    text = got.get_words_result()

    frame = got.read_camera_data()
    if frame is not None:
        nparr = np.frombuffer(frame, np.uint8)
        data = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if text:
            cv2.putText(data, text, (50, 40), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        _, jpeg = cv2.imencode('.jpg', data)
        clear_output(wait=True)
        display(Image(data=jpeg.tobytes()))

        return text

        

# Pose control

Located in a separate .py file!

In [ ]:
try:
    run_pose_control_inline(
    robot_ip='192.168.1.217',
    forward_speed=30,
    backward_speed=30,
    turn_speed=45,
    camera_index=0,
    model_path="yolov8n-pose.pt",
    up_margin_factor=0.6,
    down_margin_factor=0.6,
    min_conf=0.3,
    enable_robot=False,
    debounce_frames=2,
    max_frames=None)

except KeyboardInterrupt:
    print("Done")

# Face recognition

Located in a separate .py file!

In [ ]:
try:
    fr = face_rec.FaceRecognition(got)
    for faces in fr.run_recognition_inline(open_camera=False, display_output=True, min_confidence=0):
        print("Detected:", faces)

except KeyboardInterrupt:
    print("Done")


In [ ]:
try:
    fr = face_rec.WebcamFaceRecognition(known_dir="known")
    for faces in fr.run_recognition(min_confidence=50, stop_on_name='jensen'):
        print("Detected:", faces)

except KeyboardInterrupt:
    print("Done")


# Sample

In [ ]:
try:
    # Initial values
    got.screen_display_background(0)
    got.mechanical_joint_control(0, 45, 45, 500)

    # pose control
    run_pose_control_inline(
    forward_speed=30,
    backward_speed=30,
    turn_speed=45,
    camera_index=0,
    model_path="yolov8n-pose.pt",
    up_margin_factor=0.7,
    down_margin_factor=0.7,
    min_conf=0.3,
    enable_robot=True,
    debounce_frames=2,
    max_frames=None,
    got=got
    )

    # Stop light
    while True:
        color = color_rec()

        if color == "Purple":
            got.screen_display_background(2)
        elif color == "Blue":
            got.screen_display_background(8)
        elif color == "Green":
            got.screen_display_background(6)
            time.sleep(1)
            break
        else:
            got.screen_display_background(0)

    # Street signs
    roads = 0

    while True:
        line_type, x, y = line_follow(mult=0.25, speed=35)
        if line_type == 3 and y > 380:
            roads += 1
            got.mecanum_move_speed_times(0, 20, 25, 1)
            got.mecanum_turn_speed_times(2, 70, 90, 2)
            text = text_rec()
            if text == "IMDA":
                got.screen_display_background(1)
                got.mecanum_turn_speed_times(3, 70, 180, 2)
                break
            else:
                got.mecanum_turn_speed_times(3, 70, 90, 2)


    # After roads
    while True:
        line_type, x, y = line_follow(mult=0.25, speed=30)
        if line_type == 3 and y > 390:
            for _ in range(9):
                line_follow(mult=0.25, speed=25)
            got.mecanum_stop()
            break

    if roads == 1:
        got.mecanum_turn_speed_times(3, 60, 90, 2)
    elif roads == 3:
        got.mecanum_turn_speed_times(2, 60, 90, 2)

    for _ in range(7):
        line_follow(mult=0.25, speed=20)

    got.mecanum_stop()


    # Face recognition
    fr = face_rec.WebcamFaceRecognition(known_dir="known")
    for faces in fr.run_recognition(min_confidence=50, stop_on_name='jensen'):
        pass
        # print("Detected:", faces)

    got.screen_display_background(4)
    got.mecanum_move_speed_times(0, 30, 50, 1)


except KeyboardInterrupt:
    got.mecanum_stop()
    print("Done")